In [ ]:
import tensorflow as tf
import kagglehub
from datasets import load_dataset
from PIL import Image
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
dataset = load_dataset("naver-clova-ix/cord-v2", split="test")
gt = dataset[0]['ground_truth'] 
print(gt)

In [ ]:
image = dataset[0]['image']  # Sudah dalam bentuk PIL.Image
image = image.convert("RGB")  # Pastikan formatnya RGB

In [ ]:
# from tensorflow.keras.models import load_model

# # Coba load dulu modelnya
# model = load_model("/kaggle/input/obj_model/keras/default/1/obj_resnet50v2_v2.h5", compile=False)

# # Lihat input dan output model
# model.summary()

# # Cek bentuk input yang diminta
# print("\nModel Inputs:")
# print(model.input)

# # Cek bentuk output
# print("\nModel Outputs:")
# print(model.output)


In [ ]:
import tensorflow.keras.backend as K

if isinstance(image, Image.Image):
    image_np = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
else:
    image_np = image.copy()


def focal_loss(gamma=2., alpha=0.25):
    def loss(y_true, y_pred):
        labels = y_true
        logits = y_pred
        ce = K.binary_crossentropy(labels, logits)
        p_t = labels * logits + (1 - labels) * (1 - logits)
        alpha_factor = labels * alpha + (1 - labels) * (1 - alpha)
        modulating_factor = K.pow(1.0 - p_t, gamma)
        return K.sum(alpha_factor * modulating_factor * ce, axis=-1)
    return loss

def smooth_l1_loss(y_true, y_pred, delta=1.0):
    diff = y_true - y_pred
    abs_diff = K.abs(diff)
    loss = tf.where(abs_diff < delta, 0.5 * diff ** 2 / delta, abs_diff - 0.5 * delta)
    return K.sum(loss, axis=-1)


# Load the model with custom_objects
model_path = '/kaggle/input/obj_model/keras/default/1/obj_resnet50v2_v2.h5'
bbox_pred, class_pred = load_model(
    model_path,
    custom_objects={
        'focal_loss': focal_loss,  # TANPA KURUNG
        'smooth_l1_loss': smooth_l1_loss
    },
    compile=False
)


In [ ]:
# Preprocessing gambar
image_resized = cv2.resize(processed_rgb, (640, 640))  # Resize sesuai dengan input model
image_array = img_to_array(image_resized) / 255.0  # Normalisasi
image_array = np.expand_dims(image_array, axis=0)  # Tambahkan batch dimension
        
# Masukkan gambar ke model
prediction_outputs = model.predict([image_array])